<a href="https://colab.research.google.com/github/MoohShadox/MCNN_Training/blob/main/Forest_Firest_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn import linear_model
from sklearn.model_selection import train_test_split


In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv

--2021-01-30 14:58:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25478 (25K) [application/x-httpd-php]
Saving to: ‘forestfires.csv’

forestfires.csv     100%[===================>]  24.88K  --.-KB/s    in 0.1s    

2021-01-30 14:58:12 (186 KB/s) - ‘forestfires.csv’ saved [25478/25478]



In [3]:
!ls

forestfires.csv  sample_data


In [44]:
df = pd.read_csv("forestfires.csv")
df

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [64]:
df.corr()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
X,1.000000,0.539548,0.059816,0.028743,-0.021039,-0.048384,-0.085916,0.006210,-0.051258,0.085223,0.018798,0.065387,0.063385
Y,0.539548,1.000000,0.041664,0.079630,-0.046308,0.007782,-0.101178,-0.024488,-0.024103,0.062221,-0.020341,0.033234,0.044873
month,0.059816,0.041664,1.000000,-0.027429,-0.288542,-0.167668,-0.127198,-0.190445,-0.168058,0.126535,-0.001534,-0.031268,0.013476
day,0.028743,0.079630,-0.027429,1.000000,0.043774,0.037696,0.023736,-0.043231,0.052372,-0.115550,-0.123412,-0.064400,0.016765
FFMC,-0.021039,-0.046308,-0.288542,0.043774,1.000000,0.382619,0.330512,0.531805,0.431532,-0.300995,-0.028485,0.056702,0.040122
DMC,-0.048384,0.007782,-0.167668,0.037696,0.382619,1.000000,0.682192,0.305128,0.469594,0.073795,-0.105342,0.074790,0.072994
DC,-0.085916,-0.101178,-0.127198,0.023736,0.330512,0.682192,1.000000,0.229154,0.496208,-0.039192,-0.203466,0.035861,0.049383
ISI,0.006210,-0.024488,-0.190445,-0.043231,0.531805,0.305128,0.229154,1.000000,0.394287,-0.132517,0.106826,0.067668,0.008258
temp,-0.051258,-0.024103,-0.168058,0.052372,0.431532,0.469594,0.496208,0.394287,1.000000,-0.527390,-0.227116,0.069491,0.097844
RH,0.085223,0.062221,0.126535,-0.115550,-0.300995,0.073795,-0.039192,-0.132517,-0.527390,1.000000,0.069410,0.099751,-0.075519


In [55]:
month_id = df["month"].unique()
monthtoid = lambda x:np.argmax(month_id == x)

In [56]:
df_pret = pd.DataFrame(df)
df_pret["month"] = df_pret["month"].apply(monthtoid)
df_pret

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,0,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,1,1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,1,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,0,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,0,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,2,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,2,3,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,2,3,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,2,2,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [57]:
dum = pd.get_dummies(df_pret["month"])
df_pret = df_pret.join(dum).drop("month",axis=1)

In [58]:
df_pret.drop("day",axis=1, inplace=True)
#df.rename(columns = {'0':'month_0',})

In [65]:
y = df_pret["area"].values
y.shape

(517,)

In [68]:
df_tmp = df_pret.drop("area",axis=1)
X = df_tmp.values
X.shape

(517, 22)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
print(regr.coef_)
# The mean square error
np.mean((regr.predict(X_test) - y_test)**2)

[  2.50338213   0.02188111  -0.03696546   0.1719854   -0.06697079
   0.05066402   0.52735475  -0.20817992   0.6960532   -2.95916979
 -17.83532395  19.09417683   8.2250098   16.76886546 -18.40411341
 -15.84963884  -5.47261835  -7.37398403   9.11480101  12.5501498
  17.8496371  -18.66696142]


7709.888586553488

In [72]:
def train_step_constructor(model,loss_f,optimizer):
  def train_step(batch_x, batch_y):
    p = model(batch_x)
    loss = loss_f(batch_y.view((-1,)), p.view((-1,)))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()
  return train_step


In [153]:
class Regression(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(22, 120)
        self.fc2 = nn.Linear(120, 120)
        self.o = nn.Linear(120, 1)
        self.drop1 = nn.Dropout(0.0)
        self.drop2 = nn.Dropout(0.0)
        self.batchnorm1 = nn.BatchNorm1d(120)
        self.batchnorm2 = nn.BatchNorm1d(120)


         
    def forward(self, x):
      x = self.fc1(x)
      x = self.batchnorm1(x)
      x = self.relu(x)
      x = self.drop1(x)
      
      x = self.fc2(x)
      x = self.batchnorm2(x)
      x = self.relu(x)
      x = self.drop2(x)

      return self.o(x)

In [154]:
model = Regression()
opt = torch.optim.Adam(model.parameters() ,lr=1e-1)
mse_loss = nn.MSELoss(reduction='mean')

In [155]:
step = train_step_constructor(model,mse_loss, opt)
step

<function __main__.train_step_constructor.<locals>.train_step>

In [156]:
class Fire_Dataset(Dataset):
  def __init__(self, X,y):
    self.X = X
    self.y = y

  def __getitem__(self, id):
    return (self.X[id], self.y[id])
  
  def __len__(self):
      return len(self.X)

In [157]:
train_data = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).float())

In [158]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [159]:
model = Regression()
opt = torch.optim.Adam(model.parameters() ,lr=1e-4)
mse_loss = nn.MSELoss(reduction='mean')
step = train_step_constructor(model,mse_loss, opt)

loss = []
val_loss = []

for epoch in range(1000):
  for cpt, (batch_x, batch_y) in enumerate(train_loader):
    model.train()
    l = step(batch_x, batch_y)
    loss.append(l)
    model.eval()
    p = model(X_test).float()
    eval_loss = mse_loss(p,y_test)
    val_loss.append(eval_loss)
    if(cpt%100 == 0):
      print(eval_loss)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([171])) that is different to the input size (torch.Size([171, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(8168.4165, grad_fn=<MseLossBackward>)
tensor(8087.0166, grad_fn=<MseLossBackward>)
tensor(8075.8223, grad_fn=<MseLossBackward>)
tensor(8071.3521, grad_fn=<MseLossBackward>)
tensor(8069.4751, grad_fn=<MseLossBackward>)
tensor(8067.5371, grad_fn=<MseLossBackward>)
tensor(8066.0728, grad_fn=<MseLossBackward>)
tensor(8064.8262, grad_fn=<MseLossBackward>)
tensor(8065.0220, grad_fn=<MseLossBackward>)
tensor(8063.8271, grad_fn=<MseLossBackward>)
tensor(8062.7168, grad_fn=<MseLossBackward>)
tensor(8060.9531, grad_fn=<MseLossBackward>)
tensor(8061.2246, grad_fn=<MseLossBackward>)
tensor(8060.5156, grad_fn=<MseLossBackward>)
tensor(8059.8862, grad_fn=<MseLossBackward>)
tensor(8058.9697, grad_fn=<MseLossBackward>)
tensor(8057.5220, grad_fn=<MseLossBackward>)
tensor(8057.0513, grad_fn=<MseLossBackward>)
tensor(8055.5142, grad_fn=<MseLossBackward>)
tensor(8055.4399, grad_fn=<MseLossBackward>)
tensor(8054.7686, grad_fn=<MseLossBackward>)
tensor(8054.4087, grad_fn=<MseLossBackward>)
tensor(805

KeyboardInterrupt: ignored

In [144]:
eval_loss

tensor(8015.2598, grad_fn=<MseLossBackward>)